In [ ]:
import pandas as pd

input_file_path = '/Trade data.csv'
output_file = '/RvO Output.csv'
df = pd.read_csv(input_file_path)
# Remove option trades
df = df[df['Option'] == 0]
df.to_csv(output_file, index=False)

# Convert to datetime and create new date variables if needed
df['Date_t'] = pd.to_datetime(buy['Date_t'])
df['Year'] = df['Date_t'].dt.year
df['Month'] = df['Date_t'].dt.month

# Split buys and sells
buy = df[df['Transaction_Type'] == 1]
sell = df[df['Transaction_Type'] == 2]

# Function to determine routine traders (3 year)
def determine_routine_traders_3yr(group):
    years = group['Year'].values
    months = group['Month'].values
    routine_flag = [0] * len(group)
    for i in range(len(years) - 2):
        if (years[i + 1] == years[i] + 1) and (years[i + 2] == years[i] + 2) and (months[i + 1] == months[i]) and (
                months[i + 2] == months[i]):
            for j in range(i + 3, len(group)):
                routine_flag[j] = 1
    return pd.Series(routine_flag, index=group.index)
# Function to determine routine traders (2 year)
def determine_routine_traders_2yr(group):
    years = group['Year'].values
    months = group['Month'].values
    routine_flag = [0] * len(group)
    for i in range(len(years) - 1):
        if (years[i + 1] == years[i] + 1) and (months[i + 1] == months[i]):
            for j in range(i + 2, len(group)):
                routine_flag[j] = 1
    return pd.Series(routine_flag, index=group.index)


# Apply routine classification
buy['routine_buy_3yr'] = buy.groupby('Trader_ID').apply(determine_routine_traders_3yr).fillna(0).reset_index(level=0, drop=True)
sell['routine_sell_3yr'] = sell.groupby('Trader_ID').apply(determine_routine_traders_3yr).fillna(0).reset_index(level=0, drop=True)
buy['routine_buy_2yr'] = buy.groupby('Trader_ID').apply(determine_routine_traders_2yr).fillna(0).reset_index(level=0, drop=True)
sell['routine_sell_2yr'] = sell.groupby('Trader_ID').apply(determine_routine_traders_2yr).fillna(0).reset_index(level=0, drop=True)

# Combine buy and sell DataFrames
df = pd.concat([buy, sell])

# Save to CSV
df.to_csv(output_file, index=False)